In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Flatten, Input, Concatenate, concatenate,Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder
from tqdm.keras import TqdmCallback
import tensorflow.keras.utils as utils
from datetime import datetime

import cv2
import zipfile
import shutil
import os
import glob

from tensorflow.keras.applications import Xception, InceptionV3, MobileNet, ResNet50

import random

In [2]:
import os
import pandas as pd
import glob
import chardet


# Define the file path
tom2 = "D:\\097.지능형 스마트팜 통합 데이터(토마토)\\01.데이터\\1.Training\\원천데이터\\TS_Timeseries\\tom2\\"

#파일 목록보기
print(os.listdir(os.path.join(tom2,"양액기 수집정보")))

# Define a function to detect the encoding of a file
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# Get the list of file names
tom2_file_list = glob.glob(os.path.join(tom2, "양액기 수집정보", "*"))

# Create an empty dictionary to hold the DataFrames
lp = {}

# Loop over all files in the directory
for file_path in tom2_file_list:
    # Detect the encoding of the file
    encoding = detect_encoding(file_path)

    # Open the file with the detected encoding
    with open(file_path, 'r', encoding=encoding) as f:
        # Read the first line and split it to get column names
        columns = f.readline().strip().split(';')

        # Clean up the column names by replacing double quotes
        columns = [col.replace('"', '') for col in columns]
        columns = [col.strip() for col in columns]

        # Read the rest of the file into a DataFrame
        df = pd.read_csv(f, sep=';', names=columns, header=None)

    # Remove double quotes from data
    df = df.replace({'"': ''}, regex=True)

    # Save the DataFrame in the dictionary
    # Using the filename (without extension) as the key
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    lp[file_name] = df
    
# Time column 변경
for key in lp.keys():
    lp[key]["Time"] = pd.to_datetime(lp[key]["Time"])
    lp[key]["Time"] = lp[key]["Time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    
# MERGE
# Get a sorted list of the keys (optional)
keys = sorted(list(lp.keys()))

# Use the DataFrame from the first CSV file as the starting point
merged = lp[keys[0]]

# Loop over all other DataFrames and merge them into the merged DataFrame
for key in keys[1:]:
    merged = pd.merge(merged, lp[key], on='Time', how='inner')

print(merged.head(3))

['ESS_co2.csv', 'ESS_hum.csv', 'ESS_rad.csv', 'ESS_temp.csv', 'MC_EC.csv', 'MC_pH.csv', 'MC_temp.csv', 'MC_weight_1.csv', 'MIR_diff.csv', 'MIR_temp.csv']
                  Time  co2       습도       광량     내부온도     상부온도  급액 EC  배액 EC  \
0  2021-11-24 10:32:00    0  96.1895  91.8367  16.0042  15.9369   2.91   4.95   
1  2021-11-24 10:33:00  284  96.6069  83.4555  16.0042  15.9030   2.91   4.95   
2  2021-11-24 10:34:00  285  95.7769  65.8779  16.1056  15.9369   2.91   4.95   

   pH_Supply  pH_Drain  Temp_Soil  Temp_Supply  Temp_Drain  Scale_Supply  \
0       4.89      6.29      17.56          NaN         NaN         16.18   
1       4.89      6.29      17.56          NaN         NaN         16.18   
2       4.90      6.28      17.62          NaN         NaN         16.18   

   Scale_Drain  Deviation(Diff)        BRS  Real leaves  
0         0.16             0.31  19.930000    20.240000  
1         0.16             0.92  20.270000    21.190001  
2         0.16             0.98  20.690001

In [3]:
##
"""
Temp_Supply 급액 온도
Temp_Drain 배액 온도 

"""
# 이거 수정해야돼

'\nTemp_Supply 급액 온도\nTemp_Drain 배액 온도 \n\n'

In [4]:
print(lp["MC_temp"][["Temp_Supply","Temp_Drain"]].isnull().sum())
print(len(lp["MC_temp"][["Temp_Supply","Temp_Drain"]]))

Temp_Supply    96585
Temp_Drain     96585
dtype: int64
139598


In [5]:
for col in merged.columns:
    print(f"{col}'s null : {merged[col].isnull().sum()}")

Time's null : 0
co2's null : 0
습도's null : 0
광량's null : 0
내부온도's null : 0
상부온도's null : 0
급액 EC's null : 0
배액 EC's null : 0
pH_Supply's null : 0
pH_Drain's null : 0
Temp_Soil's null : 68
Temp_Supply's null : 79463
Temp_Drain's null : 79463
Scale_Supply's null : 0
Scale_Drain's null : 0
Deviation(Diff)'s null : 0
BRS's null : 0
Real leaves's null : 0


In [6]:
null_indices = lp["MC_temp"][lp["MC_temp"]["Temp_Soil"].isnull()].index

# Expand your indices to include the preceding and following rows
indices = []
for index in null_indices:
    indices.extend([index-1, index, index+1])

from IPython.display import display
pd.set_option('display.max_rows', None)  # or any number you prefer
pd.set_option('display.max_columns', None)  # or any number you prefer

surrounding_rows = lp["MC_temp"].loc[indices]
print(surrounding_rows)

                       Time  Temp_Soil  Temp_Supply  Temp_Drain
49039   2021-11-27 15:41:00      20.25          NaN         NaN
49040   2021-11-27 15:42:00        NaN          NaN         NaN
49041   2021-11-27 15:43:00      20.25          NaN         NaN
51282   2021-11-29 06:24:00      15.44          NaN         NaN
51283   2021-11-29 06:25:00        NaN          NaN         NaN
51284   2021-11-29 06:26:00      15.44          NaN         NaN
51600   2021-11-29 11:42:00      18.81          NaN         NaN
51601   2021-11-29 11:43:00        NaN          NaN         NaN
51602   2021-11-29 11:44:00      18.75          NaN         NaN
52272   2021-11-29 22:54:00      17.06          NaN         NaN
52273   2021-11-29 22:55:00        NaN          NaN         NaN
52274   2021-11-29 22:56:00      17.06          NaN         NaN
54162   2021-12-01 08:58:00      16.12          NaN         NaN
54163   2021-12-01 08:59:00        NaN          NaN         NaN
54164   2021-12-01 09:00:00      16.12  

In [7]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

def fill_with_mean(df, column):
    null_indices = df[df[column].isnull()].index
    for idx in null_indices:
        # Check if the previous and next indices are in the dataframe to avoid errors
        if idx - 1 in df.index and idx + 5 in df.index:
            before_value = df.loc[idx - 5, column]
            after_value = df.loc[idx + 5, column]
            
            # Only fill the NaN if both before and after values are not NaN
            if pd.notnull(before_value) and pd.notnull(after_value):
                df.loc[idx, column] = (before_value + after_value) / 5.0
    return df

lp["MC_temp"] = fill_with_mean(lp["MC_temp"], "Temp_Soil")

In [8]:
lp["MC_temp"].isnull().sum()

Time               0
Temp_Soil          2
Temp_Supply    96585
Temp_Drain     96585
dtype: int64

In [9]:
null_indices = lp["MC_temp"][lp["MC_temp"]["Temp_Soil"].isnull()].index

# Expand your indices to include the preceding and following rows
indices = []
for index in null_indices:
    indices.extend([index-1, index, index+1])

from IPython.display import display
pd.set_option('display.max_rows', None)  # or any number you prefer
pd.set_option('display.max_columns', None)  # or any number you prefer

surrounding_rows = lp["MC_temp"].loc[indices]
print(surrounding_rows)

                      Time  Temp_Soil  Temp_Supply  Temp_Drain
84464  2021-12-23 16:54:00      8.112          NaN         NaN
84465  2021-12-23 16:55:00        NaN          NaN         NaN
84466  2021-12-23 16:56:00     20.250          NaN         NaN
84469  2021-12-23 16:59:00     20.250          NaN         NaN
84470  2021-12-23 17:01:00        NaN          NaN         NaN
84471  2021-12-23 17:02:00     20.190          NaN         NaN


In [10]:
merged = fill_with_mean(merged, "Temp_Soil")

print(merged.isnull().sum())

Time                   0
co2                    0
습도                     0
광량                     0
내부온도                   0
상부온도                   0
급액 EC                  0
배액 EC                  0
pH_Supply              0
pH_Drain               0
Temp_Soil              2
Temp_Supply        79463
Temp_Drain         79463
Scale_Supply           0
Scale_Drain            0
Deviation(Diff)        0
BRS                    0
Real leaves            0
dtype: int64


In [11]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79463 entries, 0 to 79462
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             79463 non-null  object 
 1   co2              79463 non-null  int64  
 2   습도               79463 non-null  float64
 3   광량               79463 non-null  float64
 4   내부온도             79463 non-null  float64
 5   상부온도             79463 non-null  float64
 6   급액 EC            79463 non-null  float64
 7   배액 EC            79463 non-null  float64
 8   pH_Supply        79463 non-null  float64
 9   pH_Drain         79463 non-null  float64
 10  Temp_Soil        79461 non-null  float64
 11  Temp_Supply      0 non-null      float64
 12  Temp_Drain       0 non-null      float64
 13  Scale_Supply     79463 non-null  float64
 14  Scale_Drain      79463 non-null  float64
 15  Deviation(Diff)  79463 non-null  float64
 16  BRS              79463 non-null  float64
 17  Real leaves 

In [12]:
env=pd.read_csv("D:\\097.지능형 스마트팜 통합 데이터(토마토)\\01.데이터\\1.Training\\원천데이터\\TS_Timeseries\\tom2\\00_(환경제어기, 양액기)\\토마토_프리바_환경데이터_2번농가.csv")
env.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31117 entries, 0 to 31116
Data columns (total 15 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   datetime                                31117 non-null  object 
 1   meas AH:Cmp 1 (g/m3)                    30941 non-null  float64
 2   meas AH:Cmp 2 (g/m3)                    30941 non-null  float64
 3   meas AH:Cmp 3 (g/m3)                    30941 non-null  float64
 4   meas CO2:Cmp 1 (ppm)                    30941 non-null  float64
 5   meas CO2:Cmp 3 (ppm)                    30941 non-null  float64
 6   meas RH:Cmp 1 (%)                       30941 non-null  float64
 7   meas RH:Cmp 2 (%)                       30941 non-null  float64
 8   meas RH:Cmp 3 (%)                       30941 non-null  float64
 9   meas grh temp:Cmp 1 (°C)                30941 non-null  float64
 10  meas grh temp:Cmp 2 (°C)                30941 non-null  fl